In [2]:
%pip install spacy pandas numpy ollama
%python -m spacy download de_core_news_sm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [3]:
import re
import json
import pandas as pd
from ollama import Client
llama_3_3 = Client(host="http://intern.schlaubox.de:11434", timeout=500)

In [4]:
messages = [
    {
        "role" : "user",
        "content" : "What is the weight of '1 Bund Petersilie'? Just output the weight in grams. Do not generate any other text."
    }
]

In [5]:
llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)

ConnectTimeout: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
llm_analysis['message']['content']

In [6]:
data = pd.read_csv("../task2/nutrition_failed.csv")
print(data)
data

     Unnamed: 0                       ingredient       amount  \
0            15             bio hähnchenschlegel       4 stk.   
1            31                     schokoriegel       1 stk.   
2            32  amarettini mandelkeksezerstoßen  2 esslöffel   
3            33                  deko schirmchen       2 stk.   
4            40         chilisoße  sambal oelek        0.5 tl   
..          ...                              ...          ...   
98          896            sellerieknolle gegart         50 g   
99          898                 spitzpaprika rot         30 g   
100         900                         meersalz      1 prise   
101         904                         speiseöl         1 el   
102         908                             salz      1 prise   

                                       ingr_annotation  \
0                     {"anzahl": 4, "einheit": "stk."}   
1                     {"anzahl": 1, "einheit": "stk."}   
2           {"anzahl": 2, "einheit": "essl\u0

,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition,status,prompt_used,recognized_name
0,15,bio hähnchenschlegel,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""H\u00e4hnchenschlegel"", ""eigenschaf...",4.0000,stück,NaN,unrecognized,4.0 g bio hähnchenschlegel,NaN
1,31,schokoriegel,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""Schokoriegel""}",1.0000,stück,NaN,unrecognized,1.0 g schokoriegel,NaN
2,32,amarettini mandelkeksezerstoßen,2 esslöffel,"{""anzahl"": 2, ""einheit"": ""essl\u00f6ffel""}","{""zutat"": ""Amarettini"", ""eigenschaft"": ""zersto...",0.0300,liter,NaN,unrecognized,0.03 g amarettini mandelkeksezerstoßen,NaN
3,33,deko schirmchen,2 stk.,"{""anzahl"": 2, ""einheit"": ""stk.""}","{""zutat"": ""Schirmchen"", ""eigenschaft"": ""Deko""}",2.0000,stück,NaN,unrecognized,2.0 g deko schirmchen,NaN
4,40,chilisoße sambal oelek,0.5 tl,"{""volumen"": 0.5, ""einheit"": ""tl""}","{""zutat"": ""Chiliso\u00dfe"", ""eigenschaft"": ""Sa...",0.0025,liter,NaN,unrecognized,0.0025 g chilisoße sambal oelek,NaN
...,...,...,...,...,...,...,...,...,...,...,...
98,896,sellerieknolle gegart,50 g,"{""gewicht"": 50, ""einheit"": ""g""}","{""zutat"": ""Sellerieknolle"", ""eigenschaft"": ""ge...",50.0000,g,NaN,unrecognized,50.0 g Sellerieknolle,NaN
99,898,spitzpaprika rot,30 g,"{""gewicht"": 30, ""einheit"": ""g""}","{""zutat"": ""Spitzpaprika"", ""eigenschaft"": ""rot""}",30.0000,g,NaN,unrecognized,30.0 g Spitzpaprika,NaN
100,900,meersalz,1 prise,"{""anzahl"": 1, ""einheit"": ""Prise"", ""zutat"": ""Me...","{""zutat"": ""Prise"", ""eigenschaft"": ""Meer""}",0.0005,liter,NaN,no_nutrition,0.0005 g meersalz,meersalz
101,904,speiseöl,1 el,"{""volumen"": 1, ""einheit"": ""el""}","{""zutat"": ""Speise\u00f6l""}",0.0150,liter,NaN,unrecognized,0.015 g speiseöl,NaN


In [ ]:
def extract_ingredients_json(ingr_txt):
    messages = [{ "role" : "user", "content" :
    f"""Your task is to identify cooking ingredients in texts and output them in the nominative singular.
If characteristics such as color, condition, processing, quality, or origin are mentioned for an ingredient, output these characteristics as well.
Generate a json object for the output.
Always include a "ingredient" as the key for the ingredient name. Also add a "properties" array for all previously mentioned properties. If none were found, do not use this key.
If you cannot find an ingredient in the text, simply write "None" instead of a JSON.

Make sure all values are in the German basic form, like in the following example:
    input: "garnelen frisch groß"
    output: ```json
        {{
            "ingredient": "Garnele",
            "properties": ["frisch", "groß"]
        }}
    ```

If the ingredient is oddly specific, use the basic supergroup for that ingredient, e.g.:
    input: "frisches seelachsfilet"
    output: ```json
        {{
            "ingredient": "Lachsfillet",
            "properties": ["frisch"]
        }}
    ```

Now identify the ingredients in this text: {ingr_txt}"""}]
    llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)
    answer = llm_analysis['message']['content']

    # Get JSON
    json_strings = re.findall(r"```json\s*(.*?)\s*```", answer, re.DOTALL)
    json_strings = [e.strip() for e in json_strings]
    json_dicts = []
    for json_str in json_strings:
        try:
            json_obj = json.loads(json_str)
            if isinstance(json_obj, list):
                json_dicts.extend(json_obj)   # add elements individually
            else:
                json_dicts.append(json_obj)   # add a single element
        except Exception as e:
            print(e)
    return json_dicts


data['ingr_json'] = data.apply(lambda row: extract_ingredients_json(row['ingredient']), axis=1)
data.to_csv("nearly.csv", index=False)
data

In [ ]:
def extract_quantities_json(amount_txt):
    messages = [{ "role" : "user", "content" :
    f"""Your task is to identify quantities, weights, and volumes in cooking ingredients in texts.
Generate a json object for the output.
Always use 'amount' as the key for quantities, 'weight' for weights, and 'volume' for volumes.
Always specify a 'unit' key for the unit found, e.g., l, g, Stück, Prise, msp, Tafel, Riegel or Esslöffel!
Do not fabricate or include any other information in the output!
If you cannot find a quantity in the text, simply write "None" instead of a JSON.
Make sure all values are in the German basic form, like in the following example.

A few examples for this is:
Example 1:
    input: "1 Teelöffel Salz"
    output: ```json
        {{
            "amount": 1,
            "unit": "teelöffel"
        }}
    ```

Example 2:
    input: "2 Müsliriegel"
    output: ```json
        {{
            "amount": "2",
            "unit": "Riegel"
        }}
    ```

Example 3:
    input: "2 Tafeln Schokolade"
    output: ```json
        {{
            "amount": "2",
            "unit": "Tafel"
        }}

Example 4:
    input: "3 Kleckse Butter"
    output: ```json
        {{
            "amount": "3",
            "unit": "Klecks"
        }}


Example 5:
    input: "1 Schuss Rapsöl"
    output: ```json
        {{
            "amount": "1",
            "unit": "Schuss"
        }}

Example 6:
    input: "3 Päckchen frische Heidelbeeren"
    output: ```json
        {{
            "amount": "3",
            "unit": "Pack"
        }}

Example 7:
    input: "1/2 Frucht Orangensaft frisch gepresst"
    output: ```json
        {{
            "amount": "0.5",
            "unit": "Stück"
        }}
    ```

Example 8:
    input: "1/2 Kopf frischer Salat"
    output: ```json
        {{
            "amount": "0.5",
            "unit": "Kopf"
        }}
    ```

Example 9:
    input: "1 Spritzer trockener Wein"
    output: ```json
        {{
            "amount": "1",
            "unit": "Spritzer"
        }}
    ```

Now identify quantities, weights, and volumes in this text: {amount_txt}"""}]
    llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)
    answer = llm_analysis['message']['content']

    # Get JSON
    json_strings = re.findall(r"```json\s*(.*?)\s*```", answer, re.DOTALL)
    json_strings = [e.strip() for e in json_strings]
    json_dicts = []
    for json_str in json_strings:
        try:
            json_obj = json.loads(json_str)
            if isinstance(json_obj, list):
                json_dicts.extend(json_obj)   # add elements individually
            else:
                json_dicts.append(json_obj)   # add a single element
        except Exception as e:
            print(e)
    return json_dicts

data['amount_json'] = data.apply(lambda row: extract_quantities_json(row['amount']), axis=1)
data.to_csv("final.csv", index=False)
data